In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
file_id = None #'<add_you_coco_dataset_dive_link>'  #Eg. '1fdgdgssytsyusrtyshgg' ensure it to be zip
file_id_weigths = None #'<trained_weights any>' #Eg. '1fdgdgsgsjusysetg' : format: model_weights_00_100.pth

In [3]:
import gdown
import shutil

In [4]:
%%bash
cd /content/drive/MyDrive/ #goto drive location
git clone https://github.com/omer770/Fine_Tune_Grounding_Dino.git
cd Fine_Tune_Grounding_Dino
cd weights
wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth #default weights
cd /content/

Cloning into 'Fine_Tune_Grounding_Dino'...


In [ ]:
if file_id:
  zip_path = gdown.download(id=file_id)
  !unzip -q {zip_path} -d /content/drive/MyDrive/Fine_Tune_Grounding_Dino/data
if file_id_weigths:
  weights_path = gdown.download(id=file_id_weigths)
  shutil.move(weights_path, '/content/drive/MyDrive/Fine_Tune_Grounding_Dino/weights')

In [10]:
%pwd

'/content'

Run at Fine_Tune_Grounding_Dino directory

In [11]:
%cd /content/drive/MyDrive/Fine_Tune_Grounding_Dino

/content/drive/MyDrive/Fine_Tune_Grounding_Dino


In [12]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 11.4 MB/s eta 0:00:00


In [13]:
!pip install -e .

Obtaining file:///content/drive/MyDrive/Fine_Tune_Grounding_Dino
  Preparing metadata (setup.py) ... done
  Running setup.py develop for groundingdino


In [14]:
from groundingdino.util.dataset import annotation_coco_2_pd_converter

In [15]:
path_2_images = '/content/drive/MyDrive/Fine_Tune_Grounding_Dino/data/images'
path_2_csv = '/content/drive/MyDrive/Fine_Tune_Grounding_Dino/data/annotations/annotation.csv'
path_2_ann_json = '/content/drive/MyDrive/Fine_Tune_Grounding_Dino/data/annotations/instances_default.json'
path_2_sub_csv = path_2_csv.replace('annotation.','annotation_sub.')
df_ann, df_all, df_shape = annotation_coco_2_pd_converter(path_2_ann_json = path_2_ann_json,path_2_images = path_2_images,
                                                  path_2_sub_csv=path_2_sub_csv,path_2_csv = path_2_csv)


In [23]:
!wget https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fpreview.redd.it%2Fqhoecxm4rlt61.jpg%3Fwidth%3D1792%26format%3Dpjpg%26auto%3Dwebp%26v%3Denabled%26s%3Dcc6087294c75e4a94860a3d6a43bac2b30a10d5d&f=1&nofb=1&ipt=6baab542c85e00a2759e2f49464cb5a59ee9105106ad51b2255aafa66c8f48ed&ipo=images

--2024-04-15 13:05:21--  https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fpreview.redd.it%2Fqhoecxm4rlt61.jpg%3Fwidth%3D1792%26format%3Dpjpg%26auto%3Dwebp%26v%3Denabled%26s%3Dcc6087294c75e4a94860a3d6a43bac2b30a10d5d
Resolving external-content.duckduckgo.com (external-content.duckduckgo.com)... 52.250.30.213
Connecting to external-content.duckduckgo.com (external-content.duckduckgo.com)|52.250.30.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘index.html?u=https:%2F%2Fpreview.redd.it%2Fqhoecxm4rlt61.jpg?width=1792&format=pjpg&auto=webp&v=enabled&s=cc6087294c75e4a94860a3d6a43bac2b30a10d5d’

index.html?u=https:     [ <=>                ] 373.88K  --.-KB/s    in 0.1s    

2024-04-15 13:05:21 (3.31 MB/s) - ‘index.html?u=https:%2F%2Fpreview.redd.it%2Fqhoecxm4rlt61.jpg?width=1792&format=pjpg&auto=webp&v=enabled&s=cc6087294c75e4a94860a3d6a43bac2b30a10d5d’ saved [382848]



In [16]:
df_all

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
3588,caravan,910,621,80,62,61594_108148_18.jpg,1024,1024
3435,caravan,356,425,76,23,61595_108268_18.jpg,1024,1024
25547,circular structure,0,507,68,90,61516_108164_18.jpg,1024,1024
4756,pool,723,561,61,101,61588_108292_18.jpg,1024,1024
7366,building,40,696,222,134,61576_108118_18.jpg,1024,1024
1178,caravan,206,101,93,38,61607_108136_18.jpg,1024,1024
13630,caravan,357,311,60,25,61548_108087_18.jpg,1024,1024
1485,caravan,571,472,22,85,61605_108260_18.jpg,1024,1024
25547,circular structure,0,507,68,90,61516_108164_18.jpg,1024,1024
13223,circular structure,977,778,46,49,61549_108164_18.jpg,1024,1024


In [17]:
df_ann

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,building,392,788,173,121,61614_108109_18.jpg,1024,1024
1,building,673,259,148,133,61614_108109_18.jpg,1024,1024
2,building,981,1007,39,16,61614_108109_18.jpg,1024,1024
3,building,494,519,116,123,61614_108109_18.jpg,1024,1024
4,building,866,136,157,111,61614_108109_18.jpg,1024,1024
5,building,760,0,32,15,61614_108109_18.jpg,1024,1024
6,building,591,832,26,27,61614_108109_18.jpg,1024,1024
7,building,916,205,107,139,61614_108109_18.jpg,1024,1024
8,building,0,831,146,130,61614_108109_18.jpg,1024,1024
9,building,29,909,161,114,61614_108109_18.jpg,1024,1024


In [18]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!python train_run.py

In [24]:
!python test_run.py

choosen weights:  weights/model_weights_01_020.pth
final text_encoder_type: bert-base-uncased
Original boxes size torch.Size([28, 4])
The unique detected phrases are {''}
NMS boxes size torch.Size([19, 4])
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
cv2_imshow(cv2.imread('results/result.jpg'))

In [ ]:
#Test
import torch
import cv2
from test_run import process_image
from pathlib import Path
from groundingdino.util.inference import load_model, load_image, predict, annotate
from google.colab.patches import cv2_imshow

weights_Dir = Path('weights/')
filePaths = [file for file in weights_Dir.iterdir() if file.name.startswith('model_weights')]
model_weights= str(filePaths[-1])
print("choosen weights: ",model_weights)

IMAGE_PATH = "/content/drive/MyDrive/Fine_Tune_Grounding_Dino/data/test_images/img_test.jpg"
TEXT_PROMPT = "building . pool . carvan .circular stucture"
BOX_TRESHOLD = 0.5
TEXT_TRESHOLD = 0.75

process_image(model_weights=model_weights,
        image_path= IMAGE_PATH,
        text_prompt=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD)
cv2_imshow(cv2.imread('results/result.jpg'))